In [549]:
import requests
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import folium
from folium.plugins import MiniMap
from time import time
import pickle

Scrap live station status data in https://bikeshare.metro.net/stations/json/

In [550]:
def get_live_station(url):
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    stations = response.json()
    
    output = []
    
    for station in stations['features']:
        dict_keys = ['kioskId', 'bikesAvailable', 'docksAvailable', 'name', 'latitude', 'longitude']
        data = {k : station['properties'][k] for k in dict_keys}
        data['time'] = pd.to_datetime('today')
        output.append(data)

    return pd.DataFrame(output)

In [551]:
live_station_df = get_live_station("https://bikeshare.metro.net/stations/json/")
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,time
0,21,6,3005,34.04850,-118.25854,7th & Flower,2019-10-08 17:51:50.100242
1,16,8,3006,34.04554,-118.25667,Olive & 8th,2019-10-08 17:51:50.118225
2,13,10,3007,34.05048,-118.25459,5th & Grand,2019-10-08 17:51:50.118354
3,6,9,3008,34.04661,-118.26273,Figueroa & 9th,2019-10-08 17:51:50.118440
4,0,22,3010,34.03705,-118.25487,11th & Maple,2019-10-08 17:51:50.118518


Remove nano-seconds in time feature

In [553]:
live_station_df['time'] = live_station_df['time'].astype('datetime64[s]')

In [555]:
# change name of time feature to ds and set it to index
# replace minutes and seconds to 0 
live_station_df.rename(columns={'time':'ds'}, inplace=True)
live_station_df.set_index('ds', inplace=True)
live_station_df.index = live_station_df.index.map(lambda x: x.replace(second=0))
live_station_df.index = live_station_df.index.map(lambda x: x.replace(minute=0))

In [556]:
live_station_df.kioskId = live_station_df.kioskId.astype(str)

In [557]:
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
ds,,,,,,
2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower
2019-10-08 17:00:00,16,8,3006,34.04554,-118.25667,Olive & 8th
2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand
2019-10-08 17:00:00,6,9,3008,34.04661,-118.26273,Figueroa & 9th
2019-10-08 17:00:00,0,22,3010,34.03705,-118.25487,11th & Maple


### Create new column that concatenated with ds and kioskid to merge it with station_pred_df1

In [558]:
live_station_df.reset_index(inplace=True)

In [559]:
live_station_df['id'] = live_station_df.ds.astype(str) + '-' + live_station_df.kioskId
live_station_df.head()

,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,id
0,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower,2019-10-08 17:00:00-3005
1,2019-10-08 17:00:00,16,8,3006,34.04554,-118.25667,Olive & 8th,2019-10-08 17:00:00-3006
2,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand,2019-10-08 17:00:00-3007
3,2019-10-08 17:00:00,6,9,3008,34.04661,-118.26273,Figueroa & 9th,2019-10-08 17:00:00-3008
4,2019-10-08 17:00:00,0,22,3010,34.03705,-118.25487,11th & Maple,2019-10-08 17:00:00-3010


## Import forecasted station_pred_df1 that stored as a pickle

In [560]:
with open('station_pred_df1.pickle', 'rb') as f:
    station_pred_df1 = pickle.load(f)

station_pred_df1.head()

,ds,station,y_hat
0,2019-10-07 00:00:00,3047.0,-0.025745
1,2019-10-07 01:00:00,3047.0,-0.034565
2,2019-10-07 02:00:00,3047.0,-0.137689
3,2019-10-07 03:00:00,3047.0,-0.253839
4,2019-10-07 04:00:00,3047.0,-0.259322


In [561]:
station_pred_df1.tail()

,ds,station,y_hat
19173,2019-10-21 02:00:00,3078.0,0.115636
19174,2019-10-21 03:00:00,3078.0,0.076395
19175,2019-10-21 04:00:00,3078.0,0.065982
19176,2019-10-21 05:00:00,3078.0,0.099515
19177,2019-10-21 06:00:00,3078.0,0.163369


In [562]:
# Take first 4 character in station feature
station_pred_df1.station = station_pred_df1.station.str[:4]

### Create unique id feature by concatenating ds and station_id

In [563]:
station_pred_df1['id'] = station_pred_df1.ds.astype(str) + '-' + station_pred_df1.station

In [564]:
station_pred_df1.tail()

,ds,station,y_hat,id
19173,2019-10-21 02:00:00,3078,0.115636,2019-10-21 02:00:00-3078
19174,2019-10-21 03:00:00,3078,0.076395,2019-10-21 03:00:00-3078
19175,2019-10-21 04:00:00,3078,0.065982,2019-10-21 04:00:00-3078
19176,2019-10-21 05:00:00,3078,0.099515,2019-10-21 05:00:00-3078
19177,2019-10-21 06:00:00,3078,0.163369,2019-10-21 06:00:00-3078


In [565]:
station_pred_df1.set_index('ds', inplace=True)

In [566]:
merge_live_pred_df = station_pred_df1.merge(live_station_df, on='id')
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.594789,2019-10-08 17:00:00-3047,2019-10-08 17:00:00,17,18,3047,34.03998,-118.26640,Pico & Flower
1,3005,3.706416,2019-10-08 17:00:00-3005,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower
2,3023,0.612045,2019-10-08 17:00:00-3023,2019-10-08 17:00:00,11,11,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.591479,2019-10-08 17:00:00-3051,2019-10-08 17:00:00,10,6,3051,34.04542,-118.25352,7th & Broadway
4,3007,1.671935,2019-10-08 17:00:00-3007,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand


In [567]:
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.594789,2019-10-08 17:00:00-3047,2019-10-08 17:00:00,17,18,3047,34.03998,-118.26640,Pico & Flower
1,3005,3.706416,2019-10-08 17:00:00-3005,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower
2,3023,0.612045,2019-10-08 17:00:00-3023,2019-10-08 17:00:00,11,11,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.591479,2019-10-08 17:00:00-3051,2019-10-08 17:00:00,10,6,3051,34.04542,-118.25352,7th & Broadway
4,3007,1.671935,2019-10-08 17:00:00-3007,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand


In [568]:
def normalize_demand_number(record):
    if record['y_hat'] >= 4.4:
        return round(record['y_hat']) * 3
    elif record['y_hat'] >= 4:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 3:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 1:
        return round(record['y_hat']) * 3
    elif record['y_hat'] >= 0.7:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 0.4:
        return 1
    else:
        return 0

In [569]:
merge_live_pred_df['demand'] = merge_live_pred_df.apply(normalize_demand_number, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand
0,3047,0.594789,2019-10-08 17:00:00-3047,2019-10-08 17:00:00,17,18,3047,34.03998,-118.26640,Pico & Flower,1
1,3005,3.706416,2019-10-08 17:00:00-3005,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower,8
2,3023,0.612045,2019-10-08 17:00:00-3023,2019-10-08 17:00:00,11,11,3023,34.05091,-118.24097,1st & Judge John Aiso,1
3,3051,0.591479,2019-10-08 17:00:00-3051,2019-10-08 17:00:00,10,6,3051,34.04542,-118.25352,7th & Broadway,1
4,3007,1.671935,2019-10-08 17:00:00-3007,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand,6
5,3075,0.779765,2019-10-08 17:00:00-3075,2019-10-08 17:00:00,10,6,3075,34.04211,-118.25619,Broadway & 9th,2
6,3066,0.637582,2019-10-08 17:00:00-3066,2019-10-08 17:00:00,12,15,3066,34.06339,-118.23616,Spring & College,1
7,3082,0.985491,2019-10-08 17:00:00-3082,2019-10-08 17:00:00,18,9,3082,34.04652,-118.23741,Traction & Rose,2
8,3064,1.088663,2019-10-08 17:00:00-3064,2019-10-08 17:00:00,3,14,3064,34.04681,-118.25698,Grand & 7th,3
9,3052,0.639761,2019-10-08 17:00:00-3052,2019-10-08 17:00:00,6,28,3052,34.05110,-118.26456,7th & Bixel,1


In [570]:
with open('end_station_pred_df.pickle', 'rb') as f:
    end_station_pred_df = pickle.load(f)

end_station_pred_df.rename(columns={'y_hat': 'ys'}, inplace=True)
end_station_pred_df.head()

,ds,station,ys
0,2019-10-06 00:00:00,3047.0,-0.147903
1,2019-10-06 01:00:00,3047.0,-0.194775
2,2019-10-06 02:00:00,3047.0,-0.307458
3,2019-10-06 03:00:00,3047.0,-0.427971
4,2019-10-06 04:00:00,3047.0,-0.469122


## Execute the same process for end_station_pred_df

In [571]:
end_station_pred_df.station = end_station_pred_df.station.str[:4]
end_station_pred_df['id'] = end_station_pred_df.ds.astype(str) + '-' + end_station_pred_df.station
end_station_pred_df.set_index('ds', inplace=True)


In [572]:
merge_live_pred_df = merge_live_pred_df.merge(end_station_pred_df[['id', 'ys']], on='id')

In [573]:
def normalize_ys_number(record):
    if record['ys'] >= 3:
        return round(record['ys']) * 2
    elif record['ys'] >= 1:
        return round(record['ys']) * 3
    elif record['ys'] >= .8:
        return round(record['ys']) * 2
    elif record['ys'] >= .5:
        return round(record['ys']) * 1
    else:
        return 0

In [574]:
merge_live_pred_df['surplus'] = merge_live_pred_df.apply(normalize_ys_number, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand,ys,surplus
0,3047,0.594789,2019-10-08 17:00:00-3047,2019-10-08 17:00:00,17,18,3047,34.03998,-118.26640,Pico & Flower,1,0.705608,1
1,3005,3.706416,2019-10-08 17:00:00-3005,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower,8,3.561503,8
2,3023,0.612045,2019-10-08 17:00:00-3023,2019-10-08 17:00:00,11,11,3023,34.05091,-118.24097,1st & Judge John Aiso,1,0.939859,2
3,3051,0.591479,2019-10-08 17:00:00-3051,2019-10-08 17:00:00,10,6,3051,34.04542,-118.25352,7th & Broadway,1,0.734252,1
4,3007,1.671935,2019-10-08 17:00:00-3007,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand,6,0.933779,2
5,3075,0.779765,2019-10-08 17:00:00-3075,2019-10-08 17:00:00,10,6,3075,34.04211,-118.25619,Broadway & 9th,2,0.837652,2
6,3066,0.637582,2019-10-08 17:00:00-3066,2019-10-08 17:00:00,12,15,3066,34.06339,-118.23616,Spring & College,1,1.058114,3
7,3082,0.985491,2019-10-08 17:00:00-3082,2019-10-08 17:00:00,18,9,3082,34.04652,-118.23741,Traction & Rose,2,2.065758,6
8,3064,1.088663,2019-10-08 17:00:00-3064,2019-10-08 17:00:00,3,14,3064,34.04681,-118.25698,Grand & 7th,3,1.418445,3
9,3052,0.639761,2019-10-08 17:00:00-3052,2019-10-08 17:00:00,6,28,3052,34.05110,-118.26456,7th & Bixel,1,0.602813,1


In [575]:
def nexthour(record):
    if record['bikesAvailable'] - record['demand'] + record['surplus'] >= 0:
        return record['bikesAvailable'] - record['demand'] + record['surplus']
    else:
        return 0

In [576]:
merge_live_pred_df['nextHour'] = merge_live_pred_df.apply(nexthour, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand,ys,surplus,nextHour
0,3047,0.594789,2019-10-08 17:00:00-3047,2019-10-08 17:00:00,17,18,3047,34.03998,-118.26640,Pico & Flower,1,0.705608,1,17
1,3005,3.706416,2019-10-08 17:00:00-3005,2019-10-08 17:00:00,21,6,3005,34.04850,-118.25854,7th & Flower,8,3.561503,8,21
2,3023,0.612045,2019-10-08 17:00:00-3023,2019-10-08 17:00:00,11,11,3023,34.05091,-118.24097,1st & Judge John Aiso,1,0.939859,2,12
3,3051,0.591479,2019-10-08 17:00:00-3051,2019-10-08 17:00:00,10,6,3051,34.04542,-118.25352,7th & Broadway,1,0.734252,1,10
4,3007,1.671935,2019-10-08 17:00:00-3007,2019-10-08 17:00:00,13,10,3007,34.05048,-118.25459,5th & Grand,6,0.933779,2,9
5,3075,0.779765,2019-10-08 17:00:00-3075,2019-10-08 17:00:00,10,6,3075,34.04211,-118.25619,Broadway & 9th,2,0.837652,2,10
6,3066,0.637582,2019-10-08 17:00:00-3066,2019-10-08 17:00:00,12,15,3066,34.06339,-118.23616,Spring & College,1,1.058114,3,14
7,3082,0.985491,2019-10-08 17:00:00-3082,2019-10-08 17:00:00,18,9,3082,34.04652,-118.23741,Traction & Rose,2,2.065758,6,22
8,3064,1.088663,2019-10-08 17:00:00-3064,2019-10-08 17:00:00,3,14,3064,34.04681,-118.25698,Grand & 7th,3,1.418445,3,3
9,3052,0.639761,2019-10-08 17:00:00-3052,2019-10-08 17:00:00,6,28,3052,34.05110,-118.26456,7th & Bixel,1,0.602813,1,6


In [577]:
# df = merge_live_pred_df.copy()
# df.to_csv('df_9_30_5pm.csv')

In [578]:
!ls

Data                             end_station_pred_df.pickle
EDA_FoliumMap.ipynb              functions.py
LICENSE                          heatmaptime_animation.html
Live_station_status.ipynb        heatmaptimes.html
README.md                        history
TimeSeries_BikeStation.ipynb     markers.html
capstone_EDA.ipynb               station_name.csv
capstone_TimeSeries-Copy1.ipynb  station_name.json
capstone_TimeSeries.ipynb        station_pred_df1.pickle
capstone_TimeSeries_hourly.ipynb


## Create Folium Map that will display available bikes,  open docks, and forecasted available bikes in next hour for each bike stations. Also add minimap using folium plugins

In [579]:
def get_base_map(df):
    minimap = MiniMap(toggle_display=True, width=300, height=150, zoom_animation=True, zoom_level_offset=-6, position='topleft')
   
    return folium.Map(location=[df.latitude.mean(),
                             df.longitude.mean()],
                   zoom_start=20,
                     tiles='OpenStreetMap').add_child(minimap)
base = get_base_map(merge_live_pred_df)
base

In [580]:
def add_station_markers(initial_map, df):
    #station location visualization
    
    out_map = initial_map
    for lat, lon, Name, Available, Opendocks, Next_hr_remaining, in zip(df['latitude'], df['longitude'], df['name'], 
                                df['bikesAvailable'], df['docksAvailable'], df['nextHour']):
        folium.Marker([lat,lon], popup=(str(Name).capitalize() + '<br>'
                                        '<br><b>Available: </b>' + str(Available) + '<br>'
                                        '<br><b>Opendocks: </b>' + str(Opendocks) + '<br>'
                                        '<br><b>Remaining Next Hour:</b> ' + str(Next_hr_remaining)),
                     icon=folium.Icon(color='green')).add_to(out_map)
    return out_map

markers = add_station_markers(base, merge_live_pred_df)
markers 

In [172]:
!ls

Data                             capstone_TimeSeries.ipynb
LICENSE                          capstone_TimeSeries_hourly.ipynb
Live_station_status.ipynb        functions.py
README.md                        markers.html
TimeSeries_BikeStation.ipynb     station_pred_df.pickle
capstone_EDA.ipynb               station_pred_df1.pickle
capstone_TimeSeries-Copy1.ipynb


### Create dictionary key as station name and values as lat & long.
### The purpose is for flask app dropdown in index page. 

In [367]:
live_station_df.groupby('name')['latitude','longitude'].apply(lambda x: x.values.tolist()).to_dict()

{'11th & Maple': [[34.03705, -118.25487]],
 '12th & Hill': [[34.03861, -118.26086]],
 '12th & Valencia': [[34.04557, -118.27419]],
 '17th St / SMC Expo Line Station': [[34.02338, -118.47964]],
 '18th & Figueroa': [[34.03568, -118.27081]],
 '18th & San Pedro': [[34.02851, -118.25667]],
 '1st & Central': [[34.0493, -118.23881]],
 '1st & Judge John Aiso': [[34.05091, -118.24097]],
 '23rd St & Flower': [[34.03027, -118.27319]],
 '25th & Vermont': [[34.03364, -118.29131]],
 '28th & Figueroa': [[34.02629, -118.27769]],
 '28th & University': [[34.02762, -118.28068]],
 '29th & Ellendale': [[34.02835, -118.28867]],
 '2nd & Figueroa': [[34.05697, -118.25359]],
 '2nd & Hill': [[34.05287, -118.24749]],
 '32nd & Figueroa': [[34.02371, -118.27917]],
 '3rd & San Pedro': [[34.04775, -118.24317]],
 '3rd & Santa Fe': [[34.04607, -118.23309]],
 '4th & Vermont': [[34.06699, -118.29088]],
 '5th & Grand': [[34.05048, -118.25459]],
 '5th & Hewitt': [[34.04169, -118.23535]],
 '7th & Berendo': [[34.05969, -118

In [350]:
d = {}
for i in df['name'].unique():
    d[i] = [{live_station_df['name'][j]: (live_station_df['latitude'][j], live_station_df['longitude'][j])} for j in 
            live_station_df[live_station_df['name']==i].index]

In [355]:
d

{'Pico & Flower': [{'Pico & Flower': (34.03998, -118.2664)}],
 '7th & Flower': [{'7th & Flower': (34.0485, -118.25854)}],
 '1st & Judge John Aiso': [{'1st & Judge John Aiso': (34.05091, -118.24097)}],
 '7th & Broadway': [{'7th & Broadway': (34.04542, -118.25352)}],
 '5th & Grand': [{'5th & Grand': (34.05048, -118.25459)}],
 'Broadway & 9th': [{'Broadway & 9th': (34.04211, -118.25619)}],
 'Spring & College': [{'Spring & College': (34.06339, -118.23616)}],
 'Traction & Rose': [{'Traction & Rose': (34.04652, -118.23741)}],
 'Grand & 7th': [{'Grand & 7th': (34.04681, -118.25698)}],
 '7th & Bixel': [{'7th & Bixel': (34.0511, -118.26456)}],
 'Main & 4th': [{'Main & 4th': (34.04885, -118.24642)}],
 'Olive & 8th': [{'Olive & 8th': (34.04554, -118.25667)}],
 'Grand & Olympic': [{'Grand & Olympic': (34.04373, -118.26014)}],
 'Figueroa & 8th': [{'Figueroa & 8th': (34.0484, -118.26095)}],
 'Hope & 6th': [{'Hope & 6th': (34.04989, -118.25588)}],
 'Main & 1st': [{'Main & 1st': (34.05194, -118.24353)

### Take station, name, latitude, longitude features and save it as csv or json format

In [466]:
station_name_df = merge_live_pred_df[['station', 'name', 'latitude', 'longitude']]
station_name_df.shape

(56, 4)

In [473]:
station_name_df.head()

,station,name,latitude,longitude
0,3047,Pico & Flower,34.03998,-118.26640
1,3005,7th & Flower,34.04850,-118.25854
2,3023,1st & Judge John Aiso,34.05091,-118.24097
3,3051,7th & Broadway,34.04542,-118.25352
4,3007,5th & Grand,34.05048,-118.25459


In [467]:
station_name_df.to_csv("station_name.csv")

In [470]:
import csv

with open('station_name.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        stations = {}
        for row in reader:
            kiosk_id= row['station']
            lat = row['latitude']
            lng = row['longitude']
            stations[kiosk_id] = (lat, lng)


In [471]:
import json

with open('station_name.json', 'w') as outfile:
    json.dump(stations, outfile)

In [472]:
!ls

Data                             df.csv
LICENSE                          df_9_26_6pm.csv
Live_station_status.ipynb        df_9_30_5pm.csv
README.md                        functions.py
TimeSeries_BikeStation.ipynb     markers.html
capstone_EDA.ipynb               station_name.csv
capstone_TimeSeries-Copy1.ipynb  station_name.json
capstone_TimeSeries.ipynb        station_pred_df.pickle
capstone_TimeSeries_hourly.ipynb station_pred_df1.pickle
